In [ ]:
!pip cache purge 
!pip install torch -f https://download.pytorch.org/whl/torch_stable.html 
#set up for mac

In [ ]:
#NN3 with batch norm., MSE/L1 loss and AdamSGD/SGD
#a useful tutorial here:
#https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html
#also helpful codes from the 2022 cohorts



import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt




class NNDataset(Dataset):
    def __init__(self, x, y):
        self.len = x.shape[0]
        self.x_data = torch.from_numpy(x)
        self.y_data = torch.from_numpy(y)
 
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
 
    def __len__(self):
        
        return self.len

#Define the NN3,use ReLU to activate

class Activation_Net(nn.Module):
    def __init__(self,in_dim,dimhidden_1,dimhidden_2,dimhidden_3,out_dim):
        super(Activation_Net,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(in_dim,dimhidden_1),nn.ReLU(True))
        self.layer2 = nn.Sequential(
            nn.Linear(n_hidden_1,dimhidden_2),nn.ReLU(True))
        self.layer3 = nn.Sequential(
            nn.Linear(n_hidden_2,dimhidden_3),nn.ReLU(True))
        self.layer4 = nn.Sequential(nn.Linear(dimhidden_3,out_dim))

    def forward(self,x):
        x_nn = self.layer1(x_nn)
        x_nn = self.layer2(x_nn)
        x_nn = self.layer3(x_nn)
        out = x_nn
        return out

#batch normalization to boost up convergence

nndataset = NNDataset(train_input, train_output)
train_loader = DataLoader(dataset=nndataset, batch_size=5000, shuffle=True, num_workers=0) 

#can change different batch size, eg 5000, 10000 or 20000

#notice when runningon mac, when increasing the num_workers to 1 it fails


#Dimensions of all layers
#input dim=P=920

net = Activation_Net(920, 32, 16, 8, 1)

if torch.cuda.is_available():
    
    net = net.cuda()

#Choice of the loss function and SGD methods

optimizer = torch.optim.Adam(net.parameters(), lr=0.005)   #alternatively, torch.optim.SGD() or other SGD
loss_func = torch.nn.MSELoss()                             #alternatively, torch.nn.L1Loss() or other norms


for epoch in range(100):
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(torch.float32)
        labels = labels.to(torch.float32)

        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
 


        y_pred = net(inputs)
        labels = labels.reshape(-1,1)

        loss = loss_func(y_pred, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
#output layer
#use data from 2008-2015 (8 years) to train

train_input, train_output, train_tlist, covardata_cht = spltdata(2008, 8)
test_input, test_output, test_tlist, _ = spltdata(2016, 1)

x_test = torch.tensor(test_input, dtype=torch.float32)


nn_pred = net.forward(x_test)
nn_pred = nn_pred.cpu().detach().numpy()

r2_nn, t1000_nn, b1000_nn = R2_full(test_output.reshape(-1,1), nn_pred, test_tlist, test_input[:,0])

print("R^2_nn3: ",r2_nn)
print("R^2_nn3_top1000: ",t1000_nn)
print("R^2_nn3_bottom1000: ",b1000_nn)